## 模擬每日指數報酬率

In [1]:
import pandas as pd
from openpyxl import load_workbook
import openpyxl

### Step1. `取得昨天的收盤資料以及市值資料`

In [2]:
# 數據是昨天算好的
Factor_Information = pd.read_excel('Base_Data.xlsx')
Factor_Information.head(3)

,年月日,證券代碼,流通在外股數,收盤價(元)
0,2025-04-24,1215 卜蜂,294790,99.1
1,2025-04-24,1231 聯華食,271322,129.5
2,2025-04-24,1232 大統益,159975,152.5


In [3]:
# 重新命名欄位名稱，避免程式執行錯誤
Factor_Information['年月日'] = pd.to_datetime(Factor_Information['年月日'], format='%Y-%m-%d')
yesterdaytime = Factor_Information['年月日'].dt.strftime('%Y-%m-%d')[0]
Factor_Information

,年月日,證券代碼,流通在外股數,收盤價(元)
0,2025-04-24,1215 卜蜂,294790,99.1
1,2025-04-24,1231 聯華食,271322,129.5
2,2025-04-24,1232 大統益,159975,152.5
3,2025-04-24,1256 鮮活果汁-KY,33842,118.0
4,2025-04-24,1264 德麥,37064,300.0
...,...,...,...,...
342,2025-04-24,9927 泰銘,167319,69.5
343,2025-04-24,9939 宏全,295786,151.0
344,2025-04-24,9941 裕融,548225,104.0
345,2025-04-24,9942 茂順,83161,113.0


In [4]:
Factor_Information.columns = ['Date','Company','Outstanding_Share',f'{yesterdaytime}_Stock_Price']

Factor_Information = Factor_Information.drop(columns=['Date'])

# 確保資料執行的型態
Factor_Information['Company'] = Factor_Information['Company'].astype(str)

# 計算市值
Factor_Information[f'{yesterdaytime}_Factor_MV'] = (Factor_Information['Outstanding_Share'] * Factor_Information[f'{yesterdaytime}_Stock_Price'])


In [5]:
# 確保昨天日期對應的 column 名稱準確
column_name = f'{yesterdaytime}_Factor_MV'

# 確認條件是否正確篩選出所需行

filter_condition = Factor_Information["Company"] == "2330 台積電"
print("台積電市值 (修改前):")
Factor_Information.loc[filter_condition]

台積電市值 (修改前):


,Company,Outstanding_Share,2025-04-24_Stock_Price,2025-04-24_Factor_MV
44,2330 台積電,25932733,864.0,2.240588e+10


In [6]:
# 如果條件正確，執行修改
Factor_Information.loc[filter_condition, column_name] *= 0.7

# 再次打印篩選後的行以檢查修改結果
print("台積電市值 (修改後):")
Factor_Information.loc[filter_condition]

台積電市值 (修改後):


,Company,Outstanding_Share,2025-04-24_Stock_Price,2025-04-24_Factor_MV
44,2330 台積電,25932733,864.0,1.568412e+10


In [7]:
Factor_Information

,Company,Outstanding_Share,2025-04-24_Stock_Price,2025-04-24_Factor_MV
0,1215 卜蜂,294790,99.1,29213689.0
1,1231 聯華食,271322,129.5,35136199.0
2,1232 大統益,159975,152.5,24396187.5
3,1256 鮮活果汁-KY,33842,118.0,3993356.0
4,1264 德麥,37064,300.0,11119200.0
...,...,...,...,...
342,9927 泰銘,167319,69.5,11628670.5
343,9939 宏全,295786,151.0,44663686.0
344,9941 裕融,548225,104.0,57015400.0
345,9942 茂順,83161,113.0,9397193.0


In [8]:
# 計算權重
Total_MV = Factor_Information[f'{yesterdaytime}_Factor_MV'].sum()
Factor_Information['Factor_Weight'] = Factor_Information[f'{yesterdaytime}_Factor_MV'] / Total_MV

# 拆分column，把Factor拆成Factor 跟 Chinese_name
Factor_Information['Factor'] = Factor_Information['Company'].str[:4]
Factor_Information['Chinese_name'] = Factor_Information['Company'].str[4:]
Factor_Information = Factor_Information.drop(columns=['Company'])

#重新排序column順序
Factor_Information = Factor_Information[['Factor','Chinese_name','Outstanding_Share',f'{yesterdaytime}_Stock_Price',f'{yesterdaytime}_Factor_MV','Factor_Weight']]

Factor_Information

,Factor,Chinese_name,Outstanding_Share,2025-04-24_Stock_Price,2025-04-24_Factor_MV,Factor_Weight
0,1215,卜蜂,294790,99.1,29213689.0,0.000770
1,1231,聯華食,271322,129.5,35136199.0,0.000926
2,1232,大統益,159975,152.5,24396187.5,0.000643
3,1256,鮮活果汁-KY,33842,118.0,3993356.0,0.000105
4,1264,德麥,37064,300.0,11119200.0,0.000293
...,...,...,...,...,...,...
342,9927,泰銘,167319,69.5,11628670.5,0.000306
343,9939,宏全,295786,151.0,44663686.0,0.001177
344,9941,裕融,548225,104.0,57015400.0,0.001502
345,9942,茂順,83161,113.0,9397193.0,0.000248


### Step2. `下載今天的未調整收盤價以及計算報酬率`
> 資料來源是Tej，要每天更新

In [9]:
All_Stock_Today_Close_Price = pd.read_excel('Tej_close_price(update_everyday)_金滿意用.xlsx',sheet_name='close_price')
All_Stock_Today_Close_Price

,證券代碼,年月日,收盤價(元)
0,1101 台泥,2025-04-25,29.35
1,1102 亞泥,2025-04-25,43.95
2,1103 嘉泥,2025-04-25,14.95
3,1104 環泥,2025-04-25,27.60
4,1108 幸福,2025-04-25,13.75
...,...,...,...
1853,9951 皇田,2025-04-25,51.40
1854,9955 佳龍,2025-04-25,33.65
1855,9958 世紀鋼,2025-04-25,173.50
1856,9960 邁達康,2025-04-25,24.60


In [10]:
# 稍微調整一下欄位，方便後續資料處理
All_Stock_Today_Close_Price['Factor'] = All_Stock_Today_Close_Price['證券代碼'].str[:4]
All_Stock_Today_Close_Price['Chinese_name'] = All_Stock_Today_Close_Price['證券代碼'].str[5:]
All_Stock_Today_Close_Price = All_Stock_Today_Close_Price.drop(columns=['證券代碼'])
All_Stock_Today_Close_Price

,年月日,收盤價(元),Factor,Chinese_name
0,2025-04-25,29.35,1101,台泥
1,2025-04-25,43.95,1102,亞泥
2,2025-04-25,14.95,1103,嘉泥
3,2025-04-25,27.60,1104,環泥
4,2025-04-25,13.75,1108,幸福
...,...,...,...,...
1853,2025-04-25,51.40,9951,皇田
1854,2025-04-25,33.65,9955,佳龍
1855,2025-04-25,173.50,9958,世紀鋼
1856,2025-04-25,24.60,9960,邁達康


In [11]:
# 重新命名，要記得加上日期，才不會搞錯
All_Stock_Today_Close_Price['年月日'] = pd.to_datetime(All_Stock_Today_Close_Price['年月日'], format='%Y-%m-%d')
todaytime = All_Stock_Today_Close_Price['年月日'].dt.strftime('%Y%m%d')[0]
All_Stock_Today_Close_Price.columns = ['Date',f'{todaytime}_Stock_Price','Factor','Chinese_name']
All_Stock_Today_Close_Price = All_Stock_Today_Close_Price.drop(columns=['Chinese_name'])

#重新排序column順序
All_Stock_Today_Close_Price = All_Stock_Today_Close_Price[['Factor','Date',f'{todaytime}_Stock_Price']]

All_Stock_Today_Close_Price

,Factor,Date,20250425_Stock_Price
0,1101,2025-04-25,29.35
1,1102,2025-04-25,43.95
2,1103,2025-04-25,14.95
3,1104,2025-04-25,27.60
4,1108,2025-04-25,13.75
...,...,...,...
1853,9951,2025-04-25,51.40
1854,9955,2025-04-25,33.65
1855,9958,2025-04-25,173.50
1856,9960,2025-04-25,24.60


> 合併需要的欄位

In [12]:
Factor_Information['Factor'] = pd.to_numeric(Factor_Information['Factor'],errors='coerce')
All_Stock_Today_Close_Price['Factor'] = pd.to_numeric(All_Stock_Today_Close_Price['Factor'],errors='coerce')

In [13]:
History_Data_df = pd.merge(Factor_Information, All_Stock_Today_Close_Price, on='Factor', how='left')
History_Data_df

,Factor,Chinese_name,Outstanding_Share,2025-04-24_Stock_Price,2025-04-24_Factor_MV,Factor_Weight,Date,20250425_Stock_Price
0,1215,卜蜂,294790,99.1,29213689.0,0.000770,2025-04-25,98.6
1,1231,聯華食,271322,129.5,35136199.0,0.000926,2025-04-25,131.0
2,1232,大統益,159975,152.5,24396187.5,0.000643,2025-04-25,152.0
3,1256,鮮活果汁-KY,33842,118.0,3993356.0,0.000105,2025-04-25,118.0
4,1264,德麥,37064,300.0,11119200.0,0.000293,2025-04-25,298.5
...,...,...,...,...,...,...,...,...
342,9927,泰銘,167319,69.5,11628670.5,0.000306,2025-04-25,70.1
343,9939,宏全,295786,151.0,44663686.0,0.001177,2025-04-25,151.5
344,9941,裕融,548225,104.0,57015400.0,0.001502,2025-04-25,107.5
345,9942,茂順,83161,113.0,9397193.0,0.000248,2025-04-25,115.5


In [14]:
# 計算我們需要的價格變動（回報），以獲得新成份股的每日回報
History_Data_df[f'{todaytime}_Net_Return'] = (History_Data_df[f'{todaytime}_Stock_Price'] / History_Data_df[f'{yesterdaytime}_Stock_Price']) - 1
History_Data_df['Percentage_of_each_stock_return'] = History_Data_df['Factor_Weight'] * History_Data_df[f'{todaytime}_Net_Return']

# 重新排列 DataFrame 的欄位
new_column_order = ['Date','Factor','Chinese_name','Outstanding_Share',f'{yesterdaytime}_Factor_MV',f'{yesterdaytime}_Stock_Price',f'{todaytime}_Stock_Price','Factor_Weight',f'{todaytime}_Net_Return','Percentage_of_each_stock_return']
History_Data_df = History_Data_df[new_column_order]

History_Data_df

,Date,Factor,Chinese_name,Outstanding_Share,2025-04-24_Factor_MV,2025-04-24_Stock_Price,20250425_Stock_Price,Factor_Weight,20250425_Net_Return,Percentage_of_each_stock_return
0,2025-04-25,1215,卜蜂,294790,29213689.0,99.1,98.6,0.000770,-0.005045,-0.000004
1,2025-04-25,1231,聯華食,271322,35136199.0,129.5,131.0,0.000926,0.011583,0.000011
2,2025-04-25,1232,大統益,159975,24396187.5,152.5,152.0,0.000643,-0.003279,-0.000002
3,2025-04-25,1256,鮮活果汁-KY,33842,3993356.0,118.0,118.0,0.000105,0.000000,0.000000
4,2025-04-25,1264,德麥,37064,11119200.0,300.0,298.5,0.000293,-0.005000,-0.000001
...,...,...,...,...,...,...,...,...,...,...
342,2025-04-25,9927,泰銘,167319,11628670.5,69.5,70.1,0.000306,0.008633,0.000003
343,2025-04-25,9939,宏全,295786,44663686.0,151.0,151.5,0.001177,0.003311,0.000004
344,2025-04-25,9941,裕融,548225,57015400.0,104.0,107.5,0.001502,0.033654,0.000051
345,2025-04-25,9942,茂順,83161,9397193.0,113.0,115.5,0.000248,0.022124,0.000005


### Step3. `儲存為 Excel 檔案，方便保存記錄.`

In [15]:
# 備份檔案
History_Data_df.to_excel(f'{todaytime}_金滿意資料紀錄.xlsx',index=False)

### Step4. `使用今日的資料將其重組為基本資訊 (供明日使用)`

In [16]:
new_columns = ['Date','Factor','Chinese_name','Outstanding_Share',f'{todaytime}_Stock_Price']

Historical_Record_df = History_Data_df[new_columns]
Historical_Record_df['Factor'] = Historical_Record_df['Factor'].astype(str)
Historical_Record_df['證券代碼'] = Historical_Record_df['Factor'] + Historical_Record_df['Chinese_name'] 

Historical_Record_df = Historical_Record_df.drop(columns=['Factor'])
Historical_Record_df = Historical_Record_df.drop(columns=['Chinese_name'])

Historical_Record_df.columns = ['年月日','流通在外股數','收盤價(元)','證券代碼']

New_order = ['年月日','證券代碼','流通在外股數','收盤價(元)']
Historical_Record_df = Historical_Record_df[New_order]
Historical_Record_df

C:\Users\joshchiang\AppData\Local\Temp\ipykernel_17264\1447464745.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Historical_Record_df['Factor'] = Historical_Record_df['Factor'].astype(str)
C:\Users\joshchiang\AppData\Local\Temp\ipykernel_17264\1447464745.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Historical_Record_df['證券代碼'] = Historical_Record_df['Factor'] + Historical_Record_df['Chinese_name']


,年月日,證券代碼,流通在外股數,收盤價(元)
0,2025-04-25,1215 卜蜂,294790,98.6
1,2025-04-25,1231 聯華食,271322,131.0
2,2025-04-25,1232 大統益,159975,152.0
3,2025-04-25,1256 鮮活果汁-KY,33842,118.0
4,2025-04-25,1264 德麥,37064,298.5
...,...,...,...,...
342,2025-04-25,9927 泰銘,167319,70.1
343,2025-04-25,9939 宏全,295786,151.5
344,2025-04-25,9941 裕融,548225,107.5
345,2025-04-25,9942 茂順,83161,115.5


In [17]:
Historical_Record_df.to_excel('Base_Data.xlsx',index=False)

### Step5. `將模擬的每日報表儲存在指定檔案的特定欄位中，以確保代碼可重複使用。`

In [18]:
Today_Index_Net_Return = History_Data_df['Percentage_of_each_stock_return'].sum()
Today_Index_Net_Return

0.02531491552732169

In [19]:
excel_file = 'Tej_close_price(update_everyday)_金滿意用.xlsx'
sheet_name = 'index_return'
column_letter = 'H'  

wb = openpyxl.load_workbook(excel_file)

sheet = wb[sheet_name]

row = 3
while sheet[f"{column_letter}{row}"].value is not None:
    row += 1

sheet[f"{column_letter}{row}"] = Today_Index_Net_Return

wb.save(excel_file)
# wb.save('//FAS2750/PGIM_TW_Investment/T_PRUMS_中小型股基金用指數值/每日模擬指數/備份檔案/Tej_close_price(update_everyday).xlsx')    # 另存到新路徑
wb.close()